In [4]:
import numpy as np
import math

import matplotlib.pyplot as plt
import cartopy.crs as crs
import cartopy.feature as cfeature
import matplotlib.colors as colors
from matplotlib.cm import get_cmap
from matplotlib import ticker
import matplotlib.gridspec as gridspec

from cartopy import config
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point

from IPython.display import Image

from tqdm import tqdm
import os 

g = 9.8

In [15]:
Year = "2000"

Months = []
for imon in range(1,2): ### shw
    Months.append(str(imon).zfill(2))
    
Days = []
for iday in range(7,28,3):
    Days.append(str(iday).zfill(2))
    
print(Year)
print(Months)
print(Days)

2000
['01']
['07', '10', '13', '16', '19', '22', '25']


In [16]:
# Months = ['12']
# Days = ['28']

In [17]:
directory = '/n/home12/hongwei/HONGWEI/lagranto_era5_0.2um/Simulation_0.2um/2000_01/'


N_head = 5 # first 5 lines are head lines, not include data
N_column = 4+2

Nx = 36
Ny = 21
Nz = 5
N_parcel = 3780

if Nx*Ny*Nz!=N_parcel: print('ERROR: parcel number is wrong!')
    
    
# filename = "traj_"+Year+Months[0]+Days[0]+"_trace.1"
# file1 = open(directory+filename, 'r')
# Lines = file1.readlines()
# print(Nt,"days")


# for DT/DZ compare:
Criterion = -2.0

In [18]:
# loop for all traj files

for month in Months:
    for day in tqdm(Days):
    
        # (1) read original data from traj files
        filename = "traj_"+Year+month+day+"_trace.1"
        file1 = open(directory+filename, 'r')
        Lines = file1.readlines()

        Nt = int( (len(Lines)-4)/N_parcel - 1 )
                
        data = np.zeros((N_parcel, Nt, N_column))

        count = 0
        # Strips the newline character
        for line in Lines:
            count += 1
    
            if count>=5:
                i = count-5
                i_parcel = math.floor( i / (Nt+1) ) # Nt time lines plue 1 empty line
                i_t = i%(Nt+1)
        
                if i_t!=0:
                    a = line.split()

                    if i_t==1:
                        if float(a[0])!=0.0: print('ERROR: first time is not 0 !!!')
                
                    data[i_parcel,i_t-1,0] = float(a[0]) # t [h]
                    data[i_parcel,i_t-1,1] = float(a[1]) # lon
                    data[i_parcel,i_t-1,2] = float(a[2]) # lat
                    data[i_parcel,i_t-1,3] = float(a[3]) # lev
                    data[i_parcel,i_t-1,4] = float(a[4]) # RHO
                    data[i_parcel,i_t-1,5] = float(a[5]) # DTDP
                
        # (2) re-shape data(N_parcel, Nt, N_column) to data2(Nz,Ny,Nx,Nt,N_column)
        data2 = np.zeros((Nz,Ny,Nx,Nt,N_column))

        for ix in range(Nx):
            for iy in range(Ny):
                for iz in range(Nz):
                    i_parcel = ix*(Ny*Nz) + iy*Nz + iz
                    data2[iz,iy,ix,:,:] = data[i_parcel,:,:]
                
                
        # (3) calculate injected tracer lifetime in the stratosphere
        lifetime   = np.zeros((Nz,Ny,Nx))
        lifetime_N = np.zeros((Nz,Ny,Nx))
        lifetime_S = np.zeros((Nz,Ny,Nx))

        
        Lon_sink = np.zeros((Nz,Ny,Nx))
        Lat_sink = np.zeros((Nz,Ny,Nx))
        Lev_sink = np.zeros((Nz,Ny,Nx))
                
        # select the min pressure that the injected particle reaches overal
        P_min = np.zeros((Nz,Ny,Nx))
        
        for iy in range(Ny):
            for iz in range(Nz):
                for ix in range(Nx):
                    Num = 0
                    Num_N = 0
                    Num_S = 0
                    for it in range(Nt-4):
                        Num = Num+1
                    
                        if data2[iz,iy,ix,it,2]>=0: # lat>=0
                            Num_N = Num_N + 1
                        else:
                            Num_S = Num_S + 1
                    
                    
                        RHO_1 = data2[iz,iy,ix,it,4]   # [kg/m3]
                        RHO_2 = data2[iz,iy,ix,it+1,4] # 
                        RHO_3 = data2[iz,iy,ix,it+2,4] # 
                        RHO_4 = data2[iz,iy,ix,it+3,4] # 
                        RHO_5 = data2[iz,iy,ix,it+4,4] # 
                    
                        DTDP_1 = data2[iz,iy,ix,it,5]   # [K/Pa]
                        DTDP_2 = data2[iz,iy,ix,it+1,5] # [K/Pa]
                        DTDP_3 = data2[iz,iy,ix,it+2,5] # [K/Pa]
                        DTDP_4 = data2[iz,iy,ix,it+3,5] # [K/Pa]
                        DTDP_5 = data2[iz,iy,ix,it+4,5] # [K/Pa]
                    
                        DTDZ_1 = -1*DTDP_1 *g*RHO_1 * 1000.0 # [K/km]
                        DTDZ_2 = -1*DTDP_2 *g*RHO_2 * 1000.0
                        DTDZ_3 = -1*DTDP_3 *g*RHO_3 * 1000.0
                        DTDZ_4 = -1*DTDP_4 *g*RHO_4 * 1000.0
                        DTDZ_5 = -1*DTDP_5 *g*RHO_5 * 1000.0
                    
                    
                        P_1 = data2[iz,iy,ix,it,3]
                        P_2 = data2[iz,iy,ix,it+1,3]
                        P_3 = data2[iz,iy,ix,it+2,3]
                    
                        if P_1>400.0: # directly identified as tropopause
                            lifetime[iz,iy,ix] = Num # [day]
                            lifetime_N[iz,iy,ix] = Num_N
                            lifetime_S[iz,iy,ix] = Num_S
                            
                            Lon_sink[iz,iy,ix] = data2[iz,iy,ix,it,1]
                            Lat_sink[iz,iy,ix] = data2[iz,iy,ix,it,2]
                            Lev_sink[iz,iy,ix] = data2[iz,iy,ix,it,3]
                            
                            P_min[iz,iy,ix] = data2[iz,iy,ix,0,3]
                            if it>1: P_min[iz,iy,ix] = np.min(data2[iz,iy,ix,0:it,3])

                            break                     
                        elif P_1>70.0: # the particle is lower than 70 hPa level
                            if DTDZ_1<Criterion and DTDZ_2<Criterion and DTDZ_3<Criterion \
                                                and DTDZ_4<Criterion and DTDZ_5<Criterion: 
                                if P_1<P_2 and P_2<P_3: # keep dropping down
                                    lifetime[iz,iy,ix] = Num # [day]
                                    lifetime_N[iz,iy,ix] = Num_N
                                    lifetime_S[iz,iy,ix] = Num_S
                                    
                                    Lon_sink[iz,iy,ix] = data2[iz,iy,ix,it,1]
                                    Lat_sink[iz,iy,ix] = data2[iz,iy,ix,it,2]
                                    Lev_sink[iz,iy,ix] = data2[iz,iy,ix,it,3]
                                    
                                    P_min[iz,iy,ix] = data2[iz,iy,ix,0,3]
                                    if it>1: P_min[iz,iy,ix] = np.min(data2[iz,iy,ix,0:it,3])
                            
                                    break 
    
        file1.close()
        
        
        # save lifetime_01[Nz,Ny,Nx,N_inject] to Lifetime01.txt
        
        with open('./Method_1st/Lifetime_'+Year+month+day+'.txt', 'w') as f:
            for ix in range(Nx):
                for iy in range(Ny):
                    for iz in range(Nz):
                                                
                        f.write(  str(lifetime[iz,iy,ix])   + ',' \
                                + str(lifetime_N[iz,iy,ix]) + ',' \
                                + str(lifetime_S[iz,iy,ix]) + ',' \
                                
                                + str(data2[iz,iy,ix,0,1])  + ',' \
                                + str(data2[iz,iy,ix,0,2])  + ',' \
                                + str(data2[iz,iy,ix,0,3])  + ',' \
                                
                                + str(Lon_sink[iz,iy,ix])  + ',' \
                                + str(Lat_sink[iz,iy,ix])  + ',' \
                                + str(Lev_sink[iz,iy,ix])  + ',' \
                                
                                + str(P_min[iz,iy,ix])  )
                        
                        f.write('\n')    
        f.close()


100%|██████████| 7/7 [09:04<00:00, 77.73s/it]


In [19]:
print('./Method_1st/Lifetime_'+Year+month+day+'.txt')

./Method_1st/Lifetime_20000125.txt


In [ ]:
Method_1st

In [11]:
11%5


1